[Hyun woo kim]  - 2018-10-13

![](https://storage.googleapis.com/kaggle-media/competitions/PUBG/PUBG%20Inlay.jpg)

## Notebook Outline
- Competition Description
- Game Description
- Variable Description
- Simple EDA 
- Feature Engineering
- LightGBM

## 1. Competiton Description

Description : So, where we droppin' boys and girls?

Battle Royale-style video games have taken the world by storm. 100 players are dropped onto an island empty-handed and must explore, scavenge, and eliminate other players until only one is left standing, all while the play zone continues to shrink.

PlayerUnknown's BattleGrounds (PUBG) has enjoyed massive popularity. With over 50 million copies sold, it's the fifth best selling game of all time, and has millions of active monthly players.

The team at [PUBG](https://www.pubg.com/) has made official game data available for the public to explore and scavenge outside of "The Blue Circle." This competition is not an official or affiliated PUBG site - Kaggle collected data made possible through the [PUBG Developer API.](https://developer.pubg.com/)

You are given over 65,000 games' worth of anonymized player data, split into training and testing sets, and asked to predict final placement from final in-game stats and initial player ratings.

What's the best strategy to win in PUBG? Should you sit in one spot and hide your way into victory, or do you need to be the top shot? Let's let the data do the talking!

## 2. Game Description

I will try this game and know what it is but I will introduce the game simply for those who do not know. This game is a kind of Survival game that goes around a certain field and makes a fight. I have attached the game video to the link below. So it would be better to understand data

Video Link : https://www.youtube.com/watch?v=rmyyeqQpHQc

In [ ]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test  = pd.read_csv('../input/test.csv')

In [ ]:
print("Train : ",df_train.shape)
print("Test : ",df_test.shape)

In [ ]:
df_train.head()

#### reduce memory

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

## 3. Variable Description

### What is difference Id, groupId, matchId ?

In the data description,   
- matchId - Integer ID to identify match. There are no matches that are in both the training and testing set.
- groupId - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.


In [ ]:
df_train[df_train['groupId']==24]

In [ ]:
len(df_train[df_train['matchId']==0])

Consider the example above. In both cases Id is different, but groupId and matchId are the same. To illustrate this, a person A with an Id 0 and a person B with an ID 136382 are friends and have a team together (groupId). Then the same match is done, so you can assume that they entered the game with the same matchId.

To put it another way, Battlegrounds (PBUGs) have a total about 100 people per game. These 100 players have the same matchId. Among them, groupId are same as 24, so you can think that they are friends  and joined the team and played together. (There are about 100 people, not necessarily 100 people.)

In [ ]:
temp = df_train[df_train['matchId']==0]['groupId'].value_counts().sort_values(ascending=False)
#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "GroupId of Match Id:0",
    xaxis=dict(
        title='groupId',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of groupId of type of MatchId 0',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

**Note :**  You can see something strange in value counts. Four people are maximum team member and I do not know what it means more than four people. 

MichaelApers metioned it.(https://www.kaggle.com/c/pubg-finish-placement-prediction/discussion/67742#400747)   
I noticed in a kernel that @chocozzz was concerned about why there are sometimes more than 4 players with the same groupId. This can be from any of three causes:

1) The match is from a custom game/event (If there are more than 8 in a group, it is almost definitely this one)

2) The API mistakenly reported two groups as placing in the same place, and in creating groupId these groups were clumped as one larger group. This is also a contributor to the difference between "numGroups" and "maxPlace" (Teams leaving the game may also contribute).

3) There is a very rare bug in the game in which more than 4 people end up in one group (I am not sure if any of these games are included in this dataset).

I hope that this can help clarify group sizes.

### Data description detail

This game is simple. Pick up your weapons, walk around, kill enemies and survive until the end. So if you look at the variables, kill and ride will come out and if you stay alive you will win.

- assists : The assists means that i don't kill enemy but help kill enemy. So when you look at the variable, there is also a kill.
In other words, if I kill the enemy? `kill +1`. but if I did not kill the enemy but helped kill the enemy?` assists + 1.`

In [ ]:
temp = df_train['assists'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='assists',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of assists',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

### Related variables with kills

- kills : Number of enemy players killed.

In [ ]:
temp = df_train['kills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='kills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of kills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

- killStreaks : Max number of enemy players killed in a short amount of time.

In [ ]:
temp = df_train['killStreaks'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='killStreaks',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of killStreaks',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

- roadKills : Number of kills while in a vehicle.

In [ ]:
temp = df_train['roadKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='roadKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of roadKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

In [ ]:
df_train['roadKills'].value_counts()

I've explained it in more detail below, but it's hard to kill if you're in a car. So I do not understand the number 42. If you die in a car, there are usually

- The player plays the game well. ( I don't it... haha )

- teamKills : Number of times this player killed a teammate.

In [ ]:
temp = df_train['teamKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='teamKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of teamKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

In [ ]:
df_train['teamKills'].value_counts()

Rationally, teamkill is hard to understand. Still, I will explain the case I saw while playing YouTube or the game.

- A team member is a friend and kills for fun.
- The team member is not  played well. so killing the member. 
- In the case of a squad, even if they are not friends, they automatically become a team. At that time, I saw that my nationality was different or I was not a friend and I was killed.
- Only act irrational for fun.

- longestKill : Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
sns.distplot(df_train['longestKill'])

There are many kinds of guns in the game. So, as you can see in the picture below, the number of times you pick up a gun is several times.

In [ ]:
temp = df_train['weaponsAcquired'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='weaponsAcquired',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of weaponsAcquired',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

Among the guns are special guns aimed at fighters at close range, and there are sniper guns that are specially designed to match enemies at long distances (The range is fixed for each gun ). So over a certain distance, all were shot by a sniper rifle.

### headshotKills - not knocked.
HeadshotKills means that a bullet hit his head and  he died `right away`.  it is important that he died right away. 
- DBNOs : Number of enemy players knocked.

DBNOs variable means Number of enemy players `knocked`. Knocked is not dead, but can not act. so if you are knocked, your colleagues can save you (`revives` variable) but if you died? you don't save... :(

- Died picture
![](https://i.ytimg.com/vi/0qSFX2SBUho/maxresdefault.jpg)


- Knocked picture
![](https://i.ytimg.com/vi/EgLRYtUqxn4/maxresdefault.jpg)

[original picture link - above](https://www.google.co.kr/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwi2pp7i4-_dAhXGE7wKHdsJBNQQjRx6BAgBEAU&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3D0qSFX2SBUho&psig=AOvVaw1JcDsctlYqqKvW_IyzuEue&ust=1538845327420883)

[original picture link - below](https://www.google.co.kr/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwj2ybrJ4-_dAhUJwbwKHcfCDlIQjRx6BAgBEAU&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DEgLRYtUqxn4&psig=AOvVaw27IBxucFCW7i3Dd55GSlSM&ust=1538845290684243)

In [ ]:
temp = df_train['headshotKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='headshotKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of headshotKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

In [ ]:
df_train['headshotKills'].value_counts()

In [ ]:
temp = df_train['DBNOs'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='DBNOs',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of DBNOs',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

### what is difference boost vs heal?
Both of these variables are items that restore health. but the boosts immediately show the effect, and the heals show the effect slowly. 

- boosts : Number of boost items used.

![boosts](https://cdn.appuals.com/wp-content/uploads/2017/10/Battlegrounds-Healing-and-Boost-Items-Guide-5.png)


- heals : Number of healing items used.

![heals](https://cdn.appuals.com/wp-content/uploads/2017/10/Battlegrounds-Healing-and-Boost-Items-Guide-1.png)

[original picture Link](https://appuals.com/playerunknowns-battlegrounds-healing-boost-items-guide/)


In [ ]:
temp = df_train['boosts'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='boosts',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of boosts',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

In [ ]:
temp = df_train['heals'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='heals',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of heals',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

### others

- damageDealt : Total damage dealt. Note: Self inflicted damage is subtracted. If it is not headshot, it does not die in one shot. So restores health by using` boosts` or `heals`. `damageDealt` means how many bullets have ever been hit.


In [ ]:
#histogram
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(df_train['damageDealt'])

- revives : Number of times this player revived teammates. I said above,  if you knock, your teammates can save you. If a team member is saved, the revives are +1.

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
df_train['revives'].value_counts().sort_values(ascending=False).plot.bar()
plt.show()

- walkDistance : Total distance traveled on foot measured in meters.

![](https://i.ytimg.com/vi/Ig_KOUqrSH8/maxresdefault.jpg)

[original picture link](https://www.google.co.kr/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwignNbp4u_dAhWFxbwKHVGICYAQjRx6BAgBEAU&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DIg_KOUqrSH8&psig=AOvVaw06O9ien8kWzTdVEG0Fki7e&ust=1538845052210260)

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
sns.distplot(df_train['walkDistance'])

- rideDistance : Total distance traveled in vehicles measured in meters. The PUBG game is so wide that it is hard to walk around. So I ride around VEHICLE as shown in the picture below.

![](http://file.gamedonga.co.kr/files/2017/04/04/pkbg089.jpg)

[original picture link](https://www.google.co.kr/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwjl7p784u_dAhVG6LwKHceWB3wQjRx6BAgBEAU&url=http%3A%2F%2Fgame.donga.com%2F86877%2F&psig=AOvVaw01B4xxH_3KBE8QpqBsFwmH&ust=1538845127758218)

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
sns.distplot(df_train['rideDistance'])

- swimDistance: Total distance traveled by swimming measured in meters. The map is wide, and there are some kind of river. 

![](https://i.ytimg.com/vi/heUNpk8XaRU/maxresdefault.jpg)

[original picture link](https://www.google.co.kr/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwjT1e3d2e_dAhWMabwKHYh2CsAQjRx6BAgBEAU&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DheUNpk8XaRU&psig=AOvVaw2FnVjz97_tbgYfeHuGycut&ust=1538842587462044)


- vehicleDestroys: Number of vehicles destroyed.

In [ ]:
temp = df_train['vehicleDestroys'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='vehicleDestroys',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of vehicleDestroys',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

In [ ]:
df_train['vehicleDestroys'].value_counts()

If you look at the above values, you will rarely destroy a vehicle. In fact, it is very natural. It is difficult to destroy the car. And there is no profit by destroying the car. Even so, the destruction of a car can be thought of in the following sense. 

- The enemy was in the car and shot the car.
- The enemy hid the car in cover and shot the car.
- He broke a car with no meaning.

The third reason is very important. When you play games, you can meet a lot of strange people.

- weaponsAcquired : Number of weapons picked up. This game is a using gun , but it does not give a gun from the beginning. So you have to go around the map and look for weapons. In the process, you can also have heals, boosts and vehicles.

![](https://t1.daumcdn.net/cfile/tistory/9904AA3359B201AA0C)

[original picture link](https://www.google.co.kr/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwjtq9eM4-_dAhUK7bwKHeOjBPMQjRx6BAgBEAU&url=http%3A%2F%2Fhogod.tistory.com%2F23&psig=AOvVaw1SqPa1ImkjsfcThfY5nfgW&ust=1538845160945955)

In [ ]:
temp = df_train['weaponsAcquired'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='weaponsAcquired',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of weaponsAcquired',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

There are many kinds of guns in the game. So, as you can see in the picture above, the number of times you pick up a gun is several times. but the big number like 76? does not mean the new gun acquired 76. just picked up the gun, dropped it and repeated it. ( Why? There is no meaning. )

## 4. Simple EDA

### 4.1 Missing Values

In [ ]:
#missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

#histogram
#missing_data = missing_data.head(20)
percent_data = percent.head(20)
percent_data.plot(kind="bar", figsize = (8,6), fontsize = 10)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
plt.title("Total Missing Value (%) in Train", fontsize = 20)

In [ ]:
#missing data
total = df_test.isnull().sum().sort_values(ascending=False)
percent = (df_test.isnull().sum()/df_test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

#histogram
#missing_data = missing_data.head(20)
percent_data = percent.head(20)
percent_data.plot(kind="bar", figsize = (8,6), fontsize = 10)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
plt.title("Total Missing Value (%) in Test", fontsize = 20)

There is not Missing Value

### 4.2 winPlacePerc (Target Value)

In [ ]:
#winPlacePerc correlation matrix
k = 10 #number of variables for heatmap
corrmat = df_train.corr() 
cols = corrmat.nlargest(k, 'winPlacePerc').index # nlargest : Return this many descending sorted values
cm = np.corrcoef(df_train[cols].values.T) # correlation 
sns.set(font_scale=1.25)
f, ax = plt.subplots(figsize=(8, 6))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 8}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

### 4.2.2 others

In [ ]:
df_train.plot(x="walkDistance",y="winPlacePerc", kind="scatter", figsize = (8,6))

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='boosts', y="winPlacePerc", data=df_train)
fig.axis(ymin=0, ymax=1);

In [ ]:
df_train.plot(x="weaponsAcquired",y="winPlacePerc", kind="scatter", figsize = (8,6))

In [ ]:
df_train.plot(x="damageDealt",y="winPlacePerc", kind="scatter", figsize = (8,6))

In [ ]:
df_train.plot(x="heals",y="winPlacePerc", kind="scatter", figsize = (8,6))

In [ ]:
df_train.plot(x="longestKill",y="winPlacePerc", kind="scatter", figsize = (8,6))

In [ ]:
df_train.plot(x="kills",y="winPlacePerc", kind="scatter", figsize = (8,6))

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='killStreaks', y="winPlacePerc", data=df_train)
fig.axis(ymin=0, ymax=1);

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='assists', y="winPlacePerc", data=df_train)
fig.axis(ymin=0, ymax=1);

## 5. Feature Engineering

### 5.1 headshot rate

In [ ]:
headshot = df_train[['kills','winPlacePerc','headshotKills']]
headshot['headshotrate'] = headshot['headshotKills']/headshot['kills']

In [ ]:
headshot.corr()
del headshot

In [ ]:
df_train['headshotrate'] = df_train['headshotKills']/df_train['kills']
df_test['headshotrate'] = df_test['headshotKills']/df_test['kills']

### 5.2 killStreak rate

In [ ]:
killStreak = df_train[['kills','winPlacePerc','killStreaks']]
killStreak['killStreakrate'] = killStreak['killStreaks']/killStreak['kills']
killStreak.corr()

- minus killStreakrate is better than killStreaks. so i delete killStreaks and use killStreakrate

In [ ]:
del killStreak
df_train['killStreakrate'] = -(df_train['killStreaks']/df_train['kills'])
df_test['killStreakrate'] = -(df_test['killStreaks']/df_test['kills'])
del df_train['killStreaks']; del df_test['killStreaks']

### 5.3 Hackers

I refer to the kernel below to create hacker's variable
https://www.kaggle.com/rejasupotaro/cheaters-and-zombies

In [ ]:
df_train['hacker_pt'] = 0
df_test['hacker_pt'] = 0

#### Aim Hacks

In [ ]:
df_train['total_Distance'] = df_train['rideDistance'] + df_train["walkDistance"] + df_train["swimDistance"]
df_test['total_Distance'] = df_test['rideDistance'] + df_test["walkDistance"] + df_test["swimDistance"]

In [ ]:
df_train[(df_train['winPlacePerc'] == 1) & (df_train['total_Distance'] < 100)].head()

I think that `id:193` is  2kill win. because he is not move place when other playes died. 

 `Id = 1777` is not move( df_train['walkDistance']+df_train['rideDistance']+df_train['swimDistance'] == 0 ). but weaponsAcquired == 9. it is very mystery. So I defined these people as hackers in the `Items/Supplies Hacks`

In [ ]:
df_train[(df_train['winPlacePerc'] == 1) & (df_train['total_Distance'] < 100)].describe()

Most of them have boosts and heals of zero.

In [ ]:
df_train[(df_train['heals'] + df_train['boosts'] < 1) & (df_train['total_Distance'] < 100) & (df_train['kills'] > 20)].describe()

In [ ]:
df_train['hacker_pt'][(df_train['heals'] + df_train['boosts'] < 1) & (df_train['total_Distance'] < 100) & (df_train['kills'] > 20)] = 1
df_test['hacker_pt'][(df_test['heals'] + df_test['boosts'] < 1) & (df_test['total_Distance'] < 100) & (df_test['kills'] > 20)] = 1

#### Items/Supplies Hacks

In [ ]:
df_train[(df_train['kills'] > 10) &(df_train['weaponsAcquired'] >= 10) & (df_train['total_Distance'] == 0)].describe()

In [ ]:
df_train['hacker_pt'][(df_train['kills'] > 10) &(df_train['weaponsAcquired'] >= 10) & (df_train['total_Distance'] == 0)] += 1
df_test['hacker_pt'][(df_test['kills'] > 10) &(df_test['weaponsAcquired'] >= 10) & (df_test['total_Distance'] == 0)] += 1

### 5.4 health Items

In [ ]:
healthitems = df_train[['heals','winPlacePerc','boosts']]
healthitems['healthitems'] = healthitems['heals'] + healthitems['boosts']
healthitems.corr()

In [ ]:
del healthitems

This is a bad variable. so don't use it

### 5.5 kills & assists 

In [ ]:
kills = df_train[['assists','winPlacePerc','kills']]
kills['kills_assists'] = (kills['kills'] + kills['assists'])
kills.corr()

so it is good. i use kills_assists and drop kills because of high corr

In [ ]:
del kills
df_train['kills_assists'] = df_train['kills'] + df_train['assists']
df_test['kills_assists'] = df_test['kills'] + df_test['assists']
del df_train['kills']; del df_test['kills']

In [ ]:
import gc
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
gc.collect()

### 5.6 statisticals feature
https://www.kaggle.com/anycode/simple-nn-baseline

In [ ]:
df_train_size = df_train.groupby(['matchId','groupId']).size().reset_index(name='group_size')
df_test_size = df_test.groupby(['matchId','groupId']).size().reset_index(name='group_size')

df_train_mean = df_train.groupby(['matchId','groupId']).mean().reset_index()
df_test_mean = df_test.groupby(['matchId','groupId']).mean().reset_index()

df_train_max = df_train.groupby(['matchId','groupId']).max().reset_index()
df_test_max = df_test.groupby(['matchId','groupId']).max().reset_index()

df_train_min = df_train.groupby(['matchId','groupId']).min().reset_index()
df_test_min = df_test.groupby(['matchId','groupId']).min().reset_index()

"""
although you are a good game player, 
but if other players of other groups in the same match is better than you, you will still get little score
so let's add the feature of each match
"""
df_train_match_mean = df_train.groupby(['matchId']).mean().reset_index()
df_test_match_mean = df_test.groupby(['matchId']).mean().reset_index()

"""
I want to add match max feature, but get error of "memory out of limit"
"""
# df_train_match_max = df_train.groupby(['matchId']).max().reset_index()
# df_test_match_max = df_test.groupby(['matchId']).max().reset_index()

df_train = pd.merge(df_train, df_train_mean, suffixes=["", "_mean"], how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_mean, suffixes=["", "_mean"], how='left', on=['matchId', 'groupId'])
del df_train_mean
del df_test_mean

df_train = pd.merge(df_train, df_train_max, suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_max, suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
del df_train_max
del df_test_max

df_train = pd.merge(df_train, df_train_min, suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_min, suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
del df_train_min
del df_test_min

df_train = pd.merge(df_train, df_train_match_mean, suffixes=["", "_match_mean"], how='left', on=['matchId'])
df_test = pd.merge(df_test, df_test_match_mean, suffixes=["", "_match_mean"], how='left', on=['matchId'])
del df_train_match_mean
del df_test_match_mean

# df_train = pd.merge(df_train, df_train_match_max, suffixes=["", "_match_max"], how='left', on=['matchId'])
# df_test = pd.merge(df_test, df_test_match_max, suffixes=["", "_match_max"], how='left', on=['matchId'])
# del df_train_match_max
# del df_test_match_max

df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_size, how='left', on=['matchId', 'groupId'])
del df_train_size
del df_test_size


gc.collect()
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
gc.collect()

## 6. LightGBM

In [ ]:
import os
import time
import gc
import warnings
warnings.filterwarnings("ignore")
# data manipulation
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
# plot
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
# model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

### columns list

In [ ]:
train_columns = list(df_test.columns)

train_idx = df_train.Id
test_idx = df_test.Id

train_columns.remove("Id")
train_columns.remove("matchId")
train_columns.remove("groupId")

In [ ]:
x_train = df_train[train_columns]
x_test = df_test[train_columns]
y_train = df_train["winPlacePerc"].astype('float')

In [ ]:
del df_train; del df_test
gc.collect()

### Predict 

In [ ]:
# LightGBM
folds = KFold(n_splits=5,random_state=6)
oof_preds = np.zeros(x_train.shape[0])
sub_preds = np.zeros(x_test.shape[0])

start = time.time()
valid_score = 0
importances = pd.DataFrame()

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(x_train, y_train)):
    trn_x, trn_y = x_train.iloc[trn_idx], y_train[trn_idx]
    val_x, val_y = x_train.iloc[val_idx], y_train[val_idx]    
    
    train_data = lgb.Dataset(data=trn_x, label=trn_y)
    valid_data = lgb.Dataset(data=val_x, label=val_y)
    
    params = {"objective" : "regression", "metric" : "mae", 'n_estimators':10000, 'early_stopping_rounds':100,
              "num_leaves" : 30, "learning_rate" : 0.3, "bagging_fraction" : 0.9,
               "bagging_seed" : 0}
    
    lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=1000) 

    imp_df = pd.DataFrame()
    imp_df['feature'] = train_columns
    imp_df['gain'] = lgb_model.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)    
    
    oof_preds[val_idx] = lgb_model.predict(val_x, num_iteration=lgb_model.best_iteration)
    oof_preds[oof_preds>1] = 1
    oof_preds[oof_preds<0] = 0
    sub_pred = lgb_model.predict(x_test, num_iteration=lgb_model.best_iteration) / folds.n_splits
    sub_pred[sub_pred>1] = 1 # should be greater or equal to 1
    sub_pred[sub_pred<0] = 0 
    sub_preds += sub_pred
    print('Fold %2d RMSE : %.6f' % (n_fold + 1, mean_absolute_error(val_y, oof_preds[val_idx])))
    valid_score += mean_absolute_error(val_y, oof_preds[val_idx])

In [ ]:
test_pred = pd.DataFrame({"Id":test_idx})
test_pred["winPlacePerc"] = sub_preds
test_pred.columns = ["Id", "winPlacePerc"]
test_pred.to_csv("lgb_base_model.csv", index=False) # submission

### Display feature importance

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)


importances['gain_log'] = np.log1p(importances['gain'])
mean_gain = importances[['gain', 'feature']].groupby('feature').mean()
importances['mean_gain'] = importances['feature'].map(mean_gain['gain'])

plt.figure(figsize=(8, 12))
sns.barplot(x='gain_log', y='feature', data=importances.sort_values('mean_gain', ascending=False))

### If there is any part of the data that you do not understand, I will answer with a comment. I will continue to add content.